<a href="https://colab.research.google.com/github/AAnirudh07/ImageCLEF-2021/blob/main/Code/TB_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""to avoid 
InternalError:  Blas GEMM launch failed : a.shape=(32, 784), b.shape=(784, 300), m=32, n=300, k=784
     [[node sequential/dense/MatMul (defined at <ipython-input-8-644622b0f8f4>:1) ]] [Op:__inference_train_function_590]
"""
import tensorflow as tf
if(False):
  physical_devices = tf.config.list_physical_devices('TPU') 
  print(physical_devices)
  tf.config.experimental.set_memory_growth(physical_devices[0], True)
from tensorflow import keras
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# New Section

In [ ]:
import random

from scipy import ndimage
import os
import zipfile
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
def Scanlist(Addr):
    List=[]
    for i  in os.walk(Addr):
        fileNames=i[2]
        return fileNames
   
#You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#GPU CHECK
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20,90,-90]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

Num GPUs Available:  0


In [ ]:



import nibabel as nib

from scipy import ndimage


def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan


def normalize(volume):
    """Normalize the volume"""
    Air = -1000
    lung = 500
    volume[volume < Air] = Air
    volume[volume > lung] = Air
    volume = (volume - Air)/1500
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img


def process_scan(path):
    """Read and resize volume"""
    print(path)
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume
def process_scan_modified(img):
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume


In [ ]:
 
def get_model(shape):
    model = keras.models.Sequential([keras.layers.Flatten(input_shape = list(shape)), 
                                 keras.layers.Dense(600, activation = "relu" ),
                                 #keras.layers.Dense(300, activation = "relu" ),
                                 keras.layers.Dense(5, activation = "sigmoid" )]) 
    # Define the model.
    return model
 
#MOdel 1
reset1=True
#model = get_model(width=128, height=128, depth=64)
model=get_model((128,128,64))
 
model.compile(
    loss="binary_crossentropy",
    optimizer="sgd",
    metrics=["acc"],
)
# memory footprint support libraries/code
 
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
 
if(len(GPUs)>0):
  gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 if(len(GPUs)>0):
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 7.2 GB  | Proc size: 11.3 GB


In [ ]:

import random
#Validation set use for testing accuracy
def Validation(DirName,batch_size=20):
    tset=pd.read_csv('/content/drive/MyDrive/ML_TB (1)/Trainsetlabel.csv')
    FileNames=Scanlist(DirName)
    random.shuffle(FileNames)
    for i in range(0,len(FileNames),batch_size):
        Fileset=FileNames[i:i+batch_size]
        scans=None
        scanLabels=None
        scanLabels= np.array(list(tset.loc[tset['FileName']==filen]['TypeOfTB'] for filen in Fileset)) 
        buff=np.ones(scanLabels.shape)
        scanLabels=scanLabels-buff
        print(scanLabels)
        scanLabels = tf.keras.utils.to_categorical(np.array(scanLabels),num_classes=5,dtype ="int32")
        scans = np.array([process_scan(os.path.join(DirName,filen)) for filen in Fileset]) 
        return (np.array(scans),scanLabels)
#Validation set generation
ValidationAddr='/content/drive/MyDrive/ML_TB (1)/VALIDATION'
x3_val,y3_val=Validation(ValidationAddr,batch_size=20)
print("Validation Processed")


[[2.]
 [3.]
 [0.]
 [1.]
 [3.]
 [3.]
 [4.]
 [0.]
 [2.]
 [2.]
 [0.]
 [3.]
 [3.]
 [0.]
 [4.]
 [1.]
 [2.]
 [1.]
 [1.]
 [1.]]
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0701.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0810.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0407.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0610.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0806.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0809.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0915.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0409.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0704.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0702.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0410.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0808.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0807.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDATION/TRN_0411.nii.gz
/content/drive/MyDrive/ML_TB (1)/VALIDA

In [ ]:
# memory footprint support libraries/code

!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
print(type(GPUs))
if(len(GPUs)>0):
  gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 if(len(GPUs)>0):
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Created wheel for gputil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=301bcd723d5716bebb6bd75ac4dfa7823dfcf32f17f70b1b9c5895211c982b54
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
<class 'list'>
Gen RAM Free: 10.3 GB  | Proc size: 7.3 GB
GPU RAM Free: 7609MB | Used: 2MB | Util   0% | Total 7611MB


In [ ]:
y1_val,x1_val=y3_val[:],x3_val[:]

In [ ]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [ ]:
# memory footprint support libraries/code
"""
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize"""
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
print(type(GPUs))
if(len(GPUs)>0):
  gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 if(len(GPUs)>0):
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

<class 'list'>
Gen RAM Free: 8.9 GB  | Proc size: 6.9 GB


In [ ]:

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image128again_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

#TRAINING
TrainAddr='D:\PLANETS\JUPFiles\data\ALL_IMAGES'
print(model.summary())
import random
def flatten(list):
    for i in list:
        for j in i:
            yield j

def Read_map_TRAIN(DirName,x_val,y_val,split_size=6,batch_size=2,epochs=10):
    batch_num=1
    if(split_size%batch_size!=0):
        print("Not all Files are used for training")
    tset=pd.read_csv('/content/drive/MyDrive/ML_TB (1)/Trainsetlabel.csv')
    FileNames=Scanlist(DirName)
    random.shuffle(FileNames)
    for i in range(0,len(FileNames)-split_size-1,split_size):
        Fileset=FileNames[i:i+split_size]
        scans = np.array([process_scan(os.path.join(DirName,filen)) for filen in Fileset])
        scanLabels= list(list(tset.loc[tset['FileName']==filen]['TypeOfTB'] for filen in Fileset))
        scanLabels=np.array(list(flatten(scanLabels)))
        buff=np.ones(scanLabels.shape)
        scanLabels=scanLabels-buff
        print(scanLabels)
        scanLabels = tf.keras.utils.to_categorical(np.array(scanLabels),num_classes=5,dtype ="int32")
        x_train,y_train=np.array(scans),scanLabels
        train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
        train_dataset = (
          train_loader.shuffle(len(x_train))
          .map(train_preprocessing)
          .batch(batch_size)
          .prefetch(2)
        )
        print("x_val:",x_val.shape)
        print("y_val:",y_val.shape)
        print("x_train:",x_train.shape)
        print("y_train:",y_train.shape)
        print(batch_num)
      
        #Till above coomon code for train and testing
        print("y_train",y_train)
        print("y1_val",y_val)
        model.fit(
           train_dataset,
        validation_data = (x_val, y_val),
        epochs=epochs,
        shuffle=True,
        batch_size=batch_size,
        verbose=2,
        callbacks=[checkpoint_cb, early_stopping_cb],
        )
        batch_num+=1

    print("END")

model.summary()
print(y1_val)
#TRAINING
TrainAddr='/content/drive/MyDrive/ML_TB/Actuallyall'
Read_map_TRAIN(DirName=TrainAddr,split_size=40,batch_size=20,epochs=20,x_val=xt,y_val=yt)#error arises when 0.2*split_size<1



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1048576)           0         
_________________________________________________________________
dense (Dense)                (None, 512)               536871424 
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 2565      
Total params: 536,873,989
Trainable params: 536,873,989
Non-trainable params: 0
_________________________________________________________________
None
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 1048576)           0         
_________________________________________________________________
dense (Dense)                (None, 512)               536871424 
_________________

In [ ]:

import random
#Testing set use for testing accuracy
def Testing(DirName,batch_size=20):
    tset=pd.read_csv('/content/drive/MyDrive/ML_TB/Trainsetlabel.csv')
    FileNames=Scanlist(DirName)
    random.shuffle(FileNames)
    for i in range(0,len(FileNames)):
        Fileset=FileNames
        scans=None
        scanLabels=None
        scanLabels= np.array(list(tset.loc[tset['FileName']==filen]['TypeOfTB'] for filen in Fileset)) 
        buff=np.ones(scanLabels.shape)
        scanLabels=scanLabels-buff
        print(scanLabels)
        scanLabels = tf.keras.utils.to_categorical(np.array(scanLabels),num_classes=5,dtype ="int32")
        scans = np.array([process_scan(os.path.join(DirName,filen)) for filen in Fileset]) 
        return (np.array(scans),scanLabels)
#Validation set generation
ValidationAddr='/content/drive/MyDrive/ML_TB/TESTS'
xt,yt=Validation(ValidationAddr)
print("pred:",[np.argmax(i)for i in model.predict(xt)])
print("real:",[np.argmax(i)for i in yt])
print("Validation Processed")


[[3.]
 [2.]
 [3.]
 [1.]
 [3.]
 [2.]
 [3.]
 [0.]
 [2.]
 [3.]
 [2.]
 [1.]
 [1.]
 [2.]
 [2.]
 [2.]
 [1.]
 [1.]
 [1.]
 [2.]]
/content/drive/MyDrive/ML_TB/TESTS/TRN_0835.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0721.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0828.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0475.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0839.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0712.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0829.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0001.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0726.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0826.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0716.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0479.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0487.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0717.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0715.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0714.nii.gz
/content/drive/MyDrive/ML_TB/TESTS/TRN_0477.nii.gz
/content/dri

In [ ]:
FileNames=Scanlist("/content/drive/MyDrive/ML_TB/TESTS")
print(len(FileNames))

56


In [ ]:
!pip install pyyaml h5py
model.save("/content/drive/MyDrive/ML_TB/my_model1_noideaacc")

INFO:tensorflow:Assets written to: /content/drive/MyDrive/ML_TB/my_model1_noideaacc/assets


In [ ]:
import pandas as pd
import numpy as np
DirName='/content/drive/MyDrive/ML_TB/REALTEST'
FileNames=Scanlist(DirName)
for i in range(0,len(FileNames)-10,10):
  Fileset=FileNames[i:i+10]
  scans = np.array([process_scan(os.path.join(DirName,filen)) for filen in Fileset])
  names=Fileset
  scanLabels=model.predict(scans)
  val=[np.argmax(i)+1 for i in scanLabels]
  df={'FileName':names,'TypeOfTB':val}
  df=pd.DataFrame(df)
  with open('/content/drive/MyDrive/ML_TB/Ans1.csv', 'a', newline='') as f:
    df.to_csv(f, header=(f.tell()==0))


/content/drive/MyDrive/ML_TB/REALTEST/TST_0381.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0382.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0383.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0384.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0386.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0385.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0387.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0388.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0389.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0390.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0391.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0392.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0393.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0395.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0394.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0396.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0397.nii.gz
/content/drive/MyDrive/ML_TB/REALTEST/TST_0398.nii.gz
/content/drive/MyDrive/ML_TB

In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('/content/drive/MyDrive/ML_TB/my_model1_noideaacc')
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 1048576)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 600)               629146200 
_________________________________________________________________
dense_3 (Dense)              (None, 5)                 3005      
Total params: 629,149,205
Trainable params: 629,149,205
Non-trainable params: 0
_________________________________________________________________


In [ ]:
import tensorflow as tf
new_model = tf.keras.models.load_model('/content/drive/MyDrive/ML_TB (1)/my_model1_point5acc')
new_model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_3 (Flatten)          (None, 1048576)           0         
_________________________________________________________________
dense_9 (Dense)              (None, 205)               214958285 
_________________________________________________________________
dense_10 (Dense)             (None, 200)               41200     
_________________________________________________________________
dense_11 (Dense)             (None, 5)                 1005      
Total params: 215,000,490
Trainable params: 215,000,490
Non-trainable params: 0
_________________________________________________________________
